In [ ]:
import numpy as np
import cv2
import time
import datetime
import SocketServer, SimpleHTTPServer, thread, ssl



count = 0
class NextVideoHandler(SimpleHTTPServer.SimpleHTTPRequestHandler):
    def do_GET(self):
        if self.path.startswith("/script.js"):
            script = "next(" + str(count) + ")";
            self.send_response(200)
            self.send_header("Content-type", "application/javascript")
            self.send_header("Content-length", len(script))
            self.end_headers()
            self.wfile.write(script)
        else:
            script = """
<html><body><pre>

// chrome://flags/#allow-insecure-localhost

wasCount = 0;
next = function(count) {
  if (count &gt; wasCount) {
    wasCount = count
    document.getElementsByClassName("ytp-next-button")[0].click() 
  }
}

var head = document.getElementsByTagName('head')[0]
var script = null;
var cnt = 0;
var iv = setInterval(function() {
  if (script) {
    head.removeChild(script);
  }
  script = document.createElement('script');
  script.src = "https://localhost:8334/script.js";
  head.appendChild(script);
}, 2000);

"""
            self.send_response(200)
            self.send_header("Content-type", "text/html")
            self.send_header("Content-length", len(script))
            self.end_headers()
            self.wfile.write(script)


class ReusableTCPServer(SocketServer.TCPServer):
    allow_reuse_address = True

httpd = ReusableTCPServer(("0.0.0.0", 8334), NextVideoHandler )
httpd.socket = ssl.wrap_socket (httpd.socket, certfile='server.pem', keyfile='server.key', server_side=True)
thread.start_new_thread(httpd.serve_forever, ())


cap = cv2.VideoCapture('/home/oleksiyp/Videos/2017-09-03:09:23:12-raw.avi')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
today = datetime.datetime.now()

# define range of purple color in HSV
lower_purple = np.array([100,150,100])
upper_purple = np.array([110,255,255])

# Create empty points array
points = []

# Get default camera window size
ret, frame = cap.read()
Height, Width = frame.shape[:2]
frame_count = 0
kernel = np.ones((3,3), np.uint8)
threshY = Height / 4
threshRadius = Height * 0.03

out1 = None
out2 = None

#out1 = cv2.VideoWriter('/home/oleksiyp/Videos/' + today.strftime('%Y-%m-%d:%H:%M:%S') + '-raw.avi', fourcc, 25.0, (Width, Height))
out2 = cv2.VideoWriter('/home/oleksiyp/Videos/' + today.strftime('%Y-%m-%d:%H:%M:%S') + '-rec.avi', fourcc, 25.0, (Width, Height))

wasMatch = False
nFrame = 0
try:
    while cap.isOpened():
        #lower_purple = np.array([ang,0,0])
        #upper_purple = np.array([ang+20,255,255])
        #ang = ang + 1
        # Capture webcame frame
        ret, frame = cap.read()
        if not ret:
            break

        if out1:
            out1.write(frame)


        blured = cv2.GaussianBlur(frame, (11, 11), 0)

        hsv_img = cv2.cvtColor(blured, cv2.COLOR_BGR2HSV)

        # Threshold the HSV image to get only blue colors
        mask = cv2.inRange(hsv_img, lower_purple, upper_purple)
        #mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        mask = cv2.dilate(mask, kernel, iterations = 5)
        mask = cv2.erode(mask, kernel, iterations = 3)

        maskedFrame = cv2.bitwise_and(frame, frame, mask=mask);

        # Find contours, OpenCV 3.X users use this line instead
        _, contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Create empty centre array to store centroid center of mass
        center = int(Height/2), int(Width/2)

        radius = 0

        match = False
        if len(contours) > 0:

            # Get the largest contour and its center 
            c = max(contours, key=cv2.contourArea)
            (x, y), radius = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            try:
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

            except:
                center =   int(Height/2), int(Width/2)


            # Allow only countors that have a larger than 15 pixel radius
            match = radius > threshRadius and x > Width / 2
            if match:
                bx,by,bw,bh = cv2.boundingRect(c)
                nFrame = nFrame + 1
                target = maskedFrame[by:by+bh, bx:bx+bw]
                target = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
                cv2.imwrite('img/r' + str(nFrame) + '.png', target);

                # Draw cirlce and leave the last center creating a trail
                cv2.circle(frame, (int(x), int(y)), int(radius),(0, 0, 255), 2)
                cv2.circle(frame, center, 5, (0, 255, 0), -1)

                # Log center points
                points.append(center)


        # loop over the set of tracked points
        if match:
            for i in range(1, len(points)):
                try:
                    cv2.line(frame, points[i - 1], points[i], (0, 255, 0), 2)
                except:
                    pass

            # Make frame count zero
            frame_count = 0
        else:
            # Count frames
            frame_count += 1

            # If we count 10 frames without object lets delete our trail
            if frame_count == 10:
                points = []
                # when frame_count reaches 20 let's clear our trail
                frame_count = 0

        cv2.line(frame, (0, threshY), (Width, threshY), (255, 255, 0), 2)
        cv2.circle(frame, (int(threshRadius), int(threshRadius)), int(threshRadius),(0, 255, 255), 2)

        tMatch = match and center[1] < threshY

        if tMatch and wasMatch != tMatch:
            count = count + 1

        wasMatch = tMatch

        # Display our object tracker
        #frame = cv2.flip(frame, 1)
        output_text = "Count: " + str(count)
        if match:
            output_text = output_text + " H: " + str(hsv_img[int(y)][int(x)][0])
            output_text = output_text + " S: " + str(hsv_img[int(y)][int(x)][1])
            output_text = output_text + " V: " + str(hsv_img[int(y)][int(x)][2])

        cv2.putText(frame, output_text, (50,100),
                    cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,127),2)

        if out2:
            out2.write(frame)

        cv2.imshow("Object Tracker", frame)

        if cv2.waitKey(1) == 10: #13 is the Enter Key
            break
finally:
    # Release camera and close any open windows
    cap.release()

    if out1:
        out1.release()
    if out2:
        out2.release()

    cv2.destroyAllWindows()
    httpd.socket.close()
